In [1]:
from pydantic import BaseModel, Field, ConfigDict
from pydantic.alias_generators import to_camel

In [2]:
class CustomBaseModel(BaseModel):
    model_config = ConfigDict(
        extra="ignore",
        alias_generator=to_camel,
        populate_by_name=True,
    )

In [3]:
from typing import Annotated

from pydantic import AfterValidator, EmailStr, Field, PastDate


SortedStringList = Annotated[list[str], AfterValidator(lambda value: sorted(value, key=str.casefold))]

class ContactInfo(CustomBaseModel):
    email: EmailStr | None = None

class PlaceInfo(CustomBaseModel):
    city: str
    country: str

class PlaceDateInfo(CustomBaseModel):
    date_: PastDate = Field(alias="date")
    place: PlaceInfo

class PersonalInfo(CustomBaseModel):
    nationality: str
    born: PlaceDateInfo

class Person(CustomBaseModel):
    first_name: str
    last_name: str
    contact_info: ContactInfo
    personal_info: PersonalInfo
    notable_students: SortedStringList = []

In [4]:
json_data = """
{
    "firstName": "David",
    "lastName": "Hilbert",
    "contactInfo": {
        "email": "d.hilbert@spectral-theory.com",
        "homePhone": {
            "countryCode": 49,
            "areaCode": 551,
            "localPhoneNumber": 123456789
        }
    },
    "personalInfo": {
        "nationality": "German",
        "born": {
            "date": "1862-01-23",
            "place": {
                "city": "Konigsberg",
                "country": "Prussia"
            }
        },
        "died": {
            "date": "1943-02-14",
            "place": {
                "city": "Gottingen",
                "country": "Germany"
            }
        }
    },
    "awards": ["Lobachevsky Prize", "Bolyai Prize", "ForMemRS"],
    "notableStudents": ["von Neumann", "Weyl", "Courant", "Zermelo"]
}
"""

In [5]:
p = Person.model_validate_json(json_data)

In [6]:
print(p.model_dump_json(by_alias=True, indent=2))

{
  "firstName": "David",
  "lastName": "Hilbert",
  "contactInfo": {
    "email": "d.hilbert@spectral-theory.com"
  },
  "personalInfo": {
    "nationality": "German",
    "born": {
      "date": "1862-01-23",
      "place": {
        "city": "Konigsberg",
        "country": "Prussia"
      }
    }
  },
  "notableStudents": [
    "Courant",
    "von Neumann",
    "Weyl",
    "Zermelo"
  ]
}


In [7]:
class CustomBaseModel(BaseModel):
    model_config = ConfigDict(
        extra="ignore",
        alias_generator=to_camel,
        populate_by_name=True,
    )

In [8]:
from datetime import datetime
from typing import Any

import pytz
from dateutil.parser import parse
from pydantic import AfterValidator, BeforeValidator, FieldSerializationInfo, field_serializer, PlainSerializer


def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

def parse_datetime(value: Any):
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value

def dt_serializer(dt, info: FieldSerializationInfo) -> datetime | str:
    if info.mode_is_json():
        return dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    return dt

DateTimeUTC = Annotated[
    datetime,
    BeforeValidator(parse_datetime),
    AfterValidator(make_utc),
    PlainSerializer(dt_serializer, when_used="unless-none")
]

In [9]:
from uuid import uuid4

class RequestInfo(CustomBaseModel):
    query_id: uuid4 = Field(default_factory=uuid4)
    execution_dt: DateTimeUTC = Field(default_factory=lambda: datetime.now(pytz.utc))
    elapsed_time_secs: float = 0

In [10]:
class ResponseBaseModel(CustomBaseModel):
    request_info: RequestInfo

In [11]:
class Users(ResponseBaseModel):
    users: list[str] = list

In [12]:
users = Users(request_info=RequestInfo(elapsed_time_secs=20), users=["One", "Two", "Three"])

In [13]:
users

Users(request_info=RequestInfo(query_id=UUID('c84214c8-796d-4769-b25e-792068421e5c'), execution_dt=datetime.datetime(2024, 12, 7, 15, 11, 12, 532435, tzinfo=<UTC>), elapsed_time_secs=20.0), users=['One', 'Two', 'Three'])